# Интерактивное обучение SQL с базой данных Northwind

## Бизнес-аналитика и управление запасами

Этот notebook поможет вам изучить SQL на примере классической бизнес-базы данных.

In [ ]:
# Импорт библиотек
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Настройки
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
# Подключение к базе данных
try:
    conn = sqlite3.connect('../data/databases/northwind.db')
    print("✅ Подключение к базе данных установлено успешно!")
except Exception as e:
    print(f"❌ Ошибка подключения: {e}")
    print("📥 Пожалуйста, загрузите базу данных с помощью скрипта download_databases.py")

In [ ]:
# Исследование структуры базы данных
print("📊 Структура базы данных Northwind:")
print("=" * 40)

tables_query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table'
ORDER BY name;
"""

tables_df = pd.read_sql_query(tables_query, conn)
print(f"\n📋 Найдено таблиц: {len(tables_df)}")
for idx, row in tables_df.iterrows():
    print(f"• {row['name']}")

# Показываем структуру ключевых таблиц
key_tables = ['Products', 'Categories', 'Suppliers', 'Customers', 'Orders']

for table in key_tables:
    print(f"\n📄 Структура таблицы {table}:")
    schema_query = f"PRAGMA table_info({table});"
    schema_df = pd.read_sql_query(schema_query, conn)
    print(schema_df[['name', 'type']].head(8))

## Упражнение 1: Анализ продуктов и категорий

Исследуем ассортимент продукции и категории товаров.

In [ ]:
# Анализ продуктов по категориям
category_analysis_query = """
SELECT 
    c.CategoryName,
    COUNT(p.ProductID) as ProductCount,
    AVG(p.UnitPrice) as AvgPrice,
    SUM(p.UnitsInStock) as TotalStock
FROM Categories c
JOIN Products p ON c.CategoryID = p.CategoryID
GROUP BY c.CategoryID
ORDER BY ProductCount DESC;
"""

category_df = pd.read_sql_query(category_analysis_query, conn)
print("🛍️ Анализ продуктов по категориям:")
print(category_df)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Количество продуктов по категориям
axes[0].barh(category_df['CategoryName'], category_df['ProductCount'])
axes[0].set_xlabel('Количество продуктов')
axes[0].set_title('Количество продуктов по категориям')
axes[0].invert_yaxis()

# Средняя цена по категориям
axes[1].barh(category_df['CategoryName'], category_df['AvgPrice'])
axes[1].set_xlabel('Средняя цена ($)')
axes[1].set_title('Средняя цена по категориям')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Упражнение 2: Управление запасами

Анализируем состояние складских запасов и выявляем проблемы.

In [ ]:
# Продукты с низким запасом
low_stock_query = """
SELECT 
    p.ProductName,
    c.CategoryName,
    p.UnitsInStock,
    p.ReorderLevel,
    p.UnitPrice,
    s.CompanyName as Supplier
FROM Products p
JOIN Categories c ON p.CategoryID = c.CategoryID
JOIN Suppliers s ON p.SupplierID = s.SupplierID
WHERE p.UnitsInStock <= p.ReorderLevel
ORDER BY p.UnitsInStock;
"""

low_stock_df = pd.read_sql_query(low_stock_query, conn)
print(f"⚠️ Продукты, требующие пополнения запасов: {len(low_stock_df)}")
print("\n📋 Топ-10 продуктов с критическим запасом:")
print(low_stock_df.head(10))

# Анализ по поставщикам
if len(low_stock_df) > 0:
    supplier_analysis = low_stock_df.groupby('Supplier').size().sort_values(ascending=False)
    print("\n🏢 Поставщики с проблемными товарами:")
    print(supplier_analysis.head())

    # Визуализация
    plt.figure(figsize=(10, 6))
    plt.bar(supplier_analysis.head(8).index, supplier_analysis.head(8).values)
    plt.title('Количество продуктов с низким запасом по поставщикам')
    plt.xlabel('Поставщик')
    plt.ylabel('Количество проблемных продуктов')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Упражнение 3: Анализ клиентов и продаж

Исследуем клиентскую базу и паттерны покупок.

In [ ]:
# Топ-10 клиентов по количеству заказов
top_customers_query = """
SELECT 
    c.CompanyName,
    c.Country,
    c.City,
    COUNT(o.OrderID) as OrderCount,
    SUM(od.Quantity * od.UnitPrice) as TotalSpent
FROM Customers c
JOIN Orders o ON c.CustomerID = o.CustomerID
JOIN "Order Details" od ON o.OrderID = od.OrderID
GROUP BY c.CustomerID
ORDER BY OrderCount DESC
LIMIT 10;
"""

top_customers_df = pd.read_sql_query(top_customers_query, conn)
print("👥 Топ-10 клиентов по количеству заказов:")
print(top_customers_df)

# Географическое распределение клиентов
country_query = """
SELECT 
    Country,
    COUNT(*) as CustomerCount
FROM Customers
GROUP BY Country
ORDER BY CustomerCount DESC;
"""

country_df = pd.read_sql_query(country_query, conn)
print("\n🌍 Распределение клиентов по странам:")
print(country_df.head(10))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Топ-8 стран по клиентам
top_countries = country_df.head(8)
axes[0].pie(top_countries['CustomerCount'], labels=top_countries['Country'], autopct='%1.1f%%')
axes[0].set_title('Распределение клиентов по странам')

# Топ-10 клиентов по тратам
axes[1].barh(top_customers_df['CompanyName'][:5], top_customers_df['TotalSpent'][:5])
axes[1].set_xlabel('Общая сумма ($))
axes[1].set_title('Топ-5 клиентов по тратам')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## Упражнение 4: Анализ поставщиков

Оцениваем поставщиков и их продукцию.

In [ ]:
# Анализ поставщиков
supplier_analysis_query = """
SELECT 
    s.CompanyName,
    s.Country,
    COUNT(p.ProductID) as ProductCount,
    AVG(p.UnitPrice) as AvgProductPrice,
    SUM(p.UnitsInStock) as TotalStock
FROM Suppliers s
JOIN Products p ON s.SupplierID = p.SupplierID
GROUP BY s.SupplierID
ORDER BY ProductCount DESC;
"""

supplier_df = pd.read_sql_query(supplier_analysis_query, conn)
print("🚚 Анализ поставщиков:")
print(supplier_df.head(10))

# Поставщики по странам
suppliers_by_country = supplier_df.groupby('Country').agg({
    'CompanyName': 'count',
    'ProductCount': 'sum',
    'AvgProductPrice': 'mean'
}).round(2)

suppliers_by_country.columns = ['Количество_Поставщиков', 'ОбщееКоличествоПродуктов', 'СредняяЦена']
suppliers_by_country = suppliers_by_country.sort_values('Количество_Поставщиков', ascending=False)

print("\n🌍 Поставщики по странам:")
print(suppliers_by_country.head(8))

# Визуализация
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Количество поставщиков по странам
top_supplier_countries = suppliers_by_country.head(6)
axes[0,0].bar(top_supplier_countries.index, top_supplier_countries['Количество_Поставщиков'])
axes[0,0].set_title('Количество поставщиков по странам')
axes[0,0].tick_params(axis='x', rotation=45)

# Общее количество продуктов по странам
axes[0,1].bar(top_supplier_countries.index, top_supplier_countries['ОбщееКоличествоПродуктов'])
axes[0,1].set_title('Общее количество продуктов по странам')
axes[0,1].tick_params(axis='x', rotation=45)

# Средняя цена по странам
axes[1,0].bar(top_supplier_countries.index, top_supplier_countries['СредняяЦена'])
axes[1,0].set_title('Средняя цена продукта по странам')
axes[1,0].tick_params(axis='x', rotation=45)

# Топ-10 поставщиков по количеству продуктов
top_suppliers = supplier_df.head(10)
axes[1,1].barh(top_suppliers['CompanyName'], top_suppliers['ProductCount'])
axes[1,1].set_title('Топ-10 поставщиков по количеству продуктов')
axes[1,1].set_xlabel('Количество продуктов')
axes[1,1].invert_yaxis()

plt.tight_layout()
plt.show()

## Практические задания:

Попробуйте выполнить следующие запросы самостоятельно:

1. Найдите 10 самых дорогих продуктов
2. Покажите заказы за последний месяц
3. Найдите сотрудников с наибольшим количеством заказов
4. Проанализируйте продажи по кварталам
5. Найдите продукты, которые никогда не заказывали

In [ ]:
# Место для ваших решений

# Пример решения задания 1:
# expensive_products_query = """
# SELECT ProductName, UnitPrice, UnitsInStock
# FROM Products
# ORDER BY UnitPrice DESC
# LIMIT 10;
# """
# expensive_products = pd.read_sql_query(expensive_products_query, conn)
# print("Топ-10 самых дорогих продуктов:")
# print(expensive_products)

In [ ]:
# Закрываем соединение
conn.close()
print("✅ Соединение закрыто")